In [78]:
import requests
import json
import pandas as pd
#import plotly.express as px
import plotly.graph_objects as go

In [96]:
config_file = "config.json"
with open(config_file, "r") as input_file:
    config = json.load(input_file)

In [79]:
# query 24 hours
start_timestamp = 1692619200 # unix epoch in utc
interval = 900 # timestep in seconds (= 15 minutes)
number_intervals = 96
query_times = [start_timestamp + i*interval for i in range(0, 96)]

In [80]:
def query_range(queries):
    opfs = {}
    for ts in queries:
        request_opf = f"{config['network_api']}/opf/{unix_time_seconds}"
        result = requests.get(request_opf)
        opfs[ts] = json.loads(result.text)
    return opfs

In [121]:
def get_consumer_lat_lon(identifier, consumers):
    return [{ 'latitude': x['latitude'], 'longitude': x['longitude']}  for x in consumers if x['identifier'] == identifier][0]

def get_network_entity_lat_lon(identifier, entities):
    return [{ 'latitude': x['latitude'], 'longitude': x['longitude']} for x in entities if x['identifier'] == identifier][0]

def get_line_connections(consumers, network) -> pd.DataFrame:
    latitudes = []
    longitudes = []
    identifiers = []

    consumer_ids = [c['identifier'] for c in consumers]
    network_entity_ids = [n['identifier'] for n in network['entities']]
    print(consumer_ids)
    print(network_entity_ids)
    for line in network['lines']:
        fb = line['from_bus']
        tb = line['to_bus']        
        if fb in consumer_ids:
            lt = get_consumer_lat_lon(fb, consumers)
            longitudes.append(lt['longitude'])
            latitudes.append(lt['latitude'])
            identifiers.append(fb)
        elif fb in network_entity_ids:
            lt = get_network_entity_lat_lon(fb, network['entities'])
            longitudes.append(lt['longitude'])
            latitudes.append(lt['latitude'])
            identifiers.append(fb)
        else:
            print(f"identifier {fb} not found")
        if tb in consumer_ids:
            lt = get_consumer_lat_lon(tb, consumers)
            longitudes.append(lt['longitude'])
            latitudes.append(lt['latitude'])
            identifiers.append(tb)            
        elif tb in network_entity_ids:
            lt = get_network_entity_lat_lon(tb, network['entities'])
            longitudes.append(lt['longitude'])
            latitudes.append(lt['latitude'])
            identifiers.append(tb)            
        else:
            print(f"identifier {tb} not found")
        print(line)
    data = { 'identifier': identifiers, 'longitude': longitudes, 'latitude': latitudes }
    return pd.DataFrame(data=data)

In [122]:
lines_df = get_line_connections(consumers, network)
lines_df

['e6c8200f-d84b-499b-ad13-4313eec39ca2', '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'ec28306c-772c-4d65-9916-55c03ce61566']
['d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab']
{'from_bus': 'ec28306c-772c-4d65-9916-55c03ce61566', 'to_bus': '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}
{'from_bus': '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'to_bus': 'e6c8200f-d84b-499b-ad13-4313eec39ca2', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}
{'from_bus': 'e6c8200f-d84b-499b-ad13-4313eec39ca2', 'to_bus': 'd3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}


,identifier,longitude,latitude
0,ec28306c-772c-4d65-9916-55c03ce61566,14.5111,48.3740
1,60d36bfa-92a2-4521-95fa-ed0e95f2413f,14.5115,48.3744
2,60d36bfa-92a2-4521-95fa-ed0e95f2413f,14.5115,48.3744
3,e6c8200f-d84b-499b-ad13-4313eec39ca2,14.5123,48.3744
4,e6c8200f-d84b-499b-ad13-4313eec39ca2,14.5123,48.3744
5,d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab,14.5131,48.3744


In [203]:
# read consumer data
request_consumers = f"{config['consumer_api']}/consumer/all"
result = requests.get(request_consumers)
consumers = json.loads(result.text)
# read network data
initialize_network = f"{config['network_api']}/initialize"
request_initialize = requests.get(initialize_network)

request_network = f"{config['network_api']}/network"
result = requests.get(request_network)
network = json.loads(result.text)

unix_time_seconds = 1692619200
request_opf = f"{config['network_api']}/opf/{unix_time_seconds}"
result = requests.get(request_opf)
opf = json.loads(result.text)

# create pandas data frame
data = {}
data['name'] = [x['name'] for x in consumers]
data['identifier'] = [x['identifier'] for x in consumers]
data['latitude'] = [x['latitude'] for x in consumers]
data['longitude'] = [x['longitude'] for x in consumers]
data['category'] = [x['category'] for x in consumers]


data['name'] += ([x['name'] for x in network['entities']])
data['identifier'] += ([x['identifier'] for x in network['entities']])
data['latitude'] += ([x['latitude'] for x in network['entities']])
data['longitude'] += ([x['longitude'] for x in network['entities']])
data['category'] += [x['category'] for x in network['entities']]

node_df = pd.DataFrame(data=data)
bus_df = pd.DataFrame(data=opf['bus'])
load_df = pd.DataFrame(data=opf['load'])
line_df = pd.DataFrame(data=opf['line'])
res_bus = pd.DataFrame(data=opf['res_bus'])
res_line = pd.DataFrame(data=opf['res_line'])
res_load = pd.DataFrame(data=opf['res_load'])
res_ext_grid = pd.DataFrame(data=opf['res_ext_grid'])

In [169]:
# create load dataset
def create_load_dataframe(consumers, network, opf):
    load = opf['load']
    data = {}
    identifiers = [v for k, v in load['name'].items()]
    #print(identifiers)
    reverse_map ={ v: k for k, v in load['name'].items() }
    loads = [load['p_mw'][reverse_map[k]] for k in identifiers]
    #print(loads)
    data['identifier'] = identifiers
    data['load'] = loads
    return pd.DataFrame(data=data)


In [49]:
fig = px.scatter_mapbox(df_nodes, lat="latitude", lon="longitude", color="category",  color_continuous_scale=px.colors.cyclical.IceFire, zoom=15, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [187]:
mapbox_token = ""

fig = go.Figure()

lines_df = get_line_connections(consumers, network)
load_df = create_load_dataframe(consumers, network, opf)
load_map_df = load_df.join(df_nodes.set_index('identifier'), on='identifier')

# add lines
fig.add_trace(go.Scattermapbox(
    mode = 'lines',
    lon = lines_df['longitude'],
    lat = lines_df['latitude']
))

# nodes
fig.add_trace(go.Scattermapbox(
    mode = "markers",
    lon = load_map_df['longitude'],
    lat = load_map_df['latitude'],
    marker = go.scattermapbox.Marker(
        color="blue"
    ),
    hoverinfo = 'text',
    text = load_map_df['name'] + '<br>' + 'load: ' + load_map_df['load'].astype(str) + " MW"
))

fig.update_layout(    
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox_style = "dark",
    mapbox = {
        'center': { 'lon': df_nodes['longitude'].mean(), 'lat': df_nodes['latitude'].mean() },
        'zoom': 15,
        'style': "open-street-map"})

fig.update_geos(projection_type='mercator')

fig.show()

['e6c8200f-d84b-499b-ad13-4313eec39ca2', '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'ec28306c-772c-4d65-9916-55c03ce61566']
['d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab']
{'from_bus': 'ec28306c-772c-4d65-9916-55c03ce61566', 'to_bus': '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}
{'from_bus': '60d36bfa-92a2-4521-95fa-ed0e95f2413f', 'to_bus': 'e6c8200f-d84b-499b-ad13-4313eec39ca2', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}
{'from_bus': 'e6c8200f-d84b-499b-ad13-4313eec39ca2', 'to_bus': 'd3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab', 'std_type': 'NAYY 4x50 SE', 'length_km': 0.1}


In [205]:
node_df

,name,identifier,latitude,longitude,category
0,location 1,e6c8200f-d84b-499b-ad13-4313eec39ca2,48.3744,14.5123,household
1,location 2,60d36bfa-92a2-4521-95fa-ed0e95f2413f,48.3744,14.5115,household
2,location 3,ec28306c-772c-4d65-9916-55c03ce61566,48.3740,14.5111,household
3,grid connection,d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab,48.3744,14.5131,network


In [206]:
nodes_df

,name,identifier,latitude,longitude,category
0,location 1,e6c8200f-d84b-499b-ad13-4313eec39ca2,48.3744,14.5123,household
1,location 2,60d36bfa-92a2-4521-95fa-ed0e95f2413f,48.3744,14.5115,household
2,location 3,ec28306c-772c-4d65-9916-55c03ce61566,48.3740,14.5111,household
3,grid connection,d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab,48.3744,14.5131,network


In [193]:
opf['res_ext_grid']

{'p_mw': {'0': 0.019533816}, 'q_mvar': {'0': 0.1518177392}}

In [173]:
lines_df

,identifier,longitude,latitude
0,ec28306c-772c-4d65-9916-55c03ce61566,14.5111,48.3740
1,60d36bfa-92a2-4521-95fa-ed0e95f2413f,14.5115,48.3744
2,60d36bfa-92a2-4521-95fa-ed0e95f2413f,14.5115,48.3744
3,e6c8200f-d84b-499b-ad13-4313eec39ca2,14.5123,48.3744
4,e6c8200f-d84b-499b-ad13-4313eec39ca2,14.5123,48.3744
5,d3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab,14.5131,48.3744


In [174]:
load_df

,identifier,load
0,e6c8200f-d84b-499b-ad13-4313eec39ca2,0.002865
1,60d36bfa-92a2-4521-95fa-ed0e95f2413f,0.002349
2,ec28306c-772c-4d65-9916-55c03ce61566,0.000235


In [180]:
load_map_df['load_str'] = str(load_map_df['load'])

In [181]:
load_map_df

,identifier,load,name,latitude,longitude,category,load_str
0,e6c8200f-d84b-499b-ad13-4313eec39ca2,0.002865,location 1,48.3744,14.5123,household,0 0.002865\n1 0.002349\n2 0.000235\nN...
1,60d36bfa-92a2-4521-95fa-ed0e95f2413f,0.002349,location 2,48.3744,14.5115,household,0 0.002865\n1 0.002349\n2 0.000235\nN...
2,ec28306c-772c-4d65-9916-55c03ce61566,0.000235,location 3,48.3740,14.5111,household,0 0.002865\n1 0.002349\n2 0.000235\nN...
